In [106]:
# File will include the working.
# Made by Sarvesh Bhatnagar
# dontpatronizeme
from dont_patronize_me import DontPatronizeMe

# Feature
import feature.basicFeatures as bf
import feature.makeWordVector as mwv

# Preprocessing
import preprocessing.basicPreProcessing as bp

# Model
import models.deepModel as dm

# Misc for model training.
from tensorflow import keras
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
import contractions


# Scores
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

import nltk
from collections import Counter

import feature.makeWordVector as mwv


def ready_data(X, y):
    X = np.array(X)
    X = X.reshape(-1, 1)
    x_rus = X
    y_rus = y
    x_rus = [item[0] for item in x_rus]
    x_rus = np.array(x_rus).astype(np.float32)
    return x_rus, y_rus


def contract_words(text):
    """
    Removes Contractations from text. i.e. are'nt -> are not
    """
    return contractions.fix(text)


def preprocess_text(text):
    """
    Should return a list of words
    """
    text = str(text)
    text = contract_words(text)
    text = text.lower()
    text = text.replace('"', "").replace(
        ",", "").replace("'", "")
    return text.split()

def get_word_size_embeddings(text_list, size):
    """
    Returns word size embeddings
    """
    embeddings = np.zeros((size,))
    ind = len(text_list) if len(text_list) < size else size 
    for i in range(0, ind):
        embeddings[i] = len(text_list[i]) + 30
    return embeddings

def get_tags(text):
    tags_p = nltk.pos_tag(text)
    return [i[1] for i in tags_p]

def get_most_common_tags(text):
    tags_p = nltk.pos_tag(text)
    tags_p = [i[1] for i in tags_p]
    tags_p = list(Counter(tags_p).most_common(3))
    tags_p = [i[0] for i in tags_p]
    return tags_p

def get_most_common_words(text):
    stopwords = {"i", "the", "and", "or", "a", "an", "is", "are", "was", "were", "be", "been", "am", "me", "my"}
    text = [i for i in text if i not in stopwords]
    words = list(Counter(text).most_common(3))
    words = [i[0] for i in words]
    return words

def combine(x,y):
    z = []
    for i in range(len(x)):
        z.append(np.concatenate((x[i],y[i])))
    return z



In [34]:
dpm = DontPatronizeMe('dataset', 'dontpatronizeme_pcl.tsv')
dpm.load_task1()
data = dpm.train_task1_df

In [35]:
process = bp.BasicPreProcessing()
data['text_split'] = data['text'].apply(preprocess_text)
wv = mwv.Word2VecModelTrainer().load_trained("word2vec.wordvectors")
basic_features = bf.BasicFeatures()
data['embeddings'] = data['text_split'].apply(
        basic_features.add_vectors, wv=wv)

In [36]:
data["text_feature"] = data['text_split'].apply(
        basic_features.get_text_feature)
data["embeddings_feature"] = data['text_feature'].apply(
        basic_features.add_vectors_multiple, wv=wv)

data["text_feature_v2"] = data['text_split'].apply(basic_features.get_text_feature, n=[1,5])

data["embeddings_feature_v2"] = data['text_feature_v2'].apply(basic_features.add_vectors_multiple, wv=wv)

data["text_feature_v3"] = data['text_split'].apply(basic_features.get_text_feature, n=[3,7])

data["embeddings_feature_v3"] = data['text_feature_v3'].apply(basic_features.add_vectors_multiple, wv=wv)

data["word_size_embeddings"] = data['text_split'].apply(get_word_size_embeddings, size=100)

data["tags_p"] = data["text_split"].apply(get_tags)
data["most_common_words"] = data["text_split"].apply(get_most_common_words)
data["most_common_tags"] = data["text_split"].apply(get_most_common_tags)
wvec = mwv.Word2VecModelTrainer(sentences=data["tags_p"], path="pos_tags.wordvectors")
# wvec.train(size=10)
wvp = wvec.load_trained("pos_tags.wordvectors")

data["pos_embeddings"] = data["tags_p"].apply(basic_features.add_vectors, wv=wvp)
data["most_common_tags_embeddings"] = data["most_common_tags"].apply(basic_features.add_vectors, wv=wvp)
data["most_common_words_embeddings"] = data["most_common_words"].apply(basic_features.add_vectors, wv=wv)




In [37]:
def combine(x,y):
    z = []
    for i in range(len(x)):
        if(type(y[i]) == int):
            print("INT", i)
        z.append(np.concatenate((x[i],y[i])))
    return z

In [38]:
data["text_split"][8639]

[]

In [39]:
# ll = combine(data["embeddings_feature"],data["pos_embeddings"])
# data["embeddings_feature"]
# data["pos_embeddings"]

In [40]:
dpm = DontPatronizeMe('dataset', 'dontpatronizeme_pcl.tsv')
dpm.load_task1()
data = dpm.train_task1_df
# Deep Learning Pipeline.
def task_one_data(data, labels=True):
    # Load the data.
    
    process = bp.BasicPreProcessing()
    data['text_split'] = data['text'].apply(preprocess_text)

    # Train WordVectors. Only run once.
    # mwv.Word2VecModelTrainer(
    #     sentences=data['text_split'], path="dataword.wordvectors").train()

    # Load the trained word vectors.
    wv = mwv.Word2VecModelTrainer().load_trained("word2vec.wordvectors")

    # Make Embedding Columns for each text split.
    basic_features = bf.BasicFeatures()
    data['embeddings'] = data['text_split'].apply(
        basic_features.add_vectors, wv=wv)

    # NOTE NEW FEATURE
    data["text_feature"] = data['text_split'].apply(
        basic_features.get_text_feature)
    
    
    data["embeddings_feature"] = data['text_feature'].apply(
        basic_features.add_vectors_multiple, wv=wv)

    data["text_feature_v2"] = data['text_split'].apply(basic_features.get_text_feature, n=[1,5])

    data["embeddings_feature_v2"] = data['text_feature_v2'].apply(basic_features.add_vectors_multiple, wv=wv)

    data["text_feature_v3"] = data['text_split'].apply(basic_features.get_text_feature, n=[3,7])

    data["embeddings_feature_v3"] = data['text_feature_v3'].apply(basic_features.add_vectors_multiple, wv=wv)

    data["word_size_embeddings"] = data['text_split'].apply(get_word_size_embeddings, size=100)
    
    data["tags_p"] = data["text_split"].apply(get_tags)
    data["most_common_words"] = data["text_split"].apply(get_most_common_words)
    data["most_common_tags"] = data["text_split"].apply(get_most_common_tags)
    wvec = mwv.Word2VecModelTrainer(sentences=data["tags_p"], path="pos_tags.wordvectors")
    # wvec.train(size=10)
    wvp = wvec.load_trained("pos_tags.wordvectors")

    data["pos_embeddings"] = data["tags_p"].apply(basic_features.add_vectors, wv=wvp)
    data["most_common_tags_embeddings"] = data["most_common_tags"].apply(basic_features.add_vectors, wv=wvp)
    data["most_common_words_embeddings"] = data["most_common_words"].apply(basic_features.add_vectors, wv=wv)


    ll = combine(data["embeddings_feature"],data["pos_embeddings"])
    # ll = combine(ll,data["most_common_tags_embeddings"])
    # ll = combine(ll,data["most_common_words_embeddings"])
    # ll = combine(ll,data["embeddings_feature_v2"])
    ll = combine(ll,data["word_size_embeddings"])
    # ll = combine(ll,data["embeddings_feature_v3"])
    # ll = combine(ll,data["embeddings"])


    data["combined"] = ll

    rus = RandomUnderSampler(random_state=42)
    X_train = data["combined"]
    if(labels):
        y_train = data["label"]
        return X_train, y_train
    return X_train

In [41]:
X_train, y_train = task_one_data(data)

In [44]:
y_train[1].shape

()

In [4]:
# rus = RandomUnderSampler(random_state=42)
# X_train, X_test, y_train, y_test = train_test_split(
#     data['combined'], data['label'], stratify=data['label'], test_size=0.2, random_state=1)


In [23]:
# data["combined"][3].shape

(210,)

In [5]:
nn_model = dm.NNModels(input_shape=X_train[0].shape,)
# TODO data["combined"][0].shape


rus = RandomOverSampler(random_state=42,sampling_strategy=1)
X_train = np.array(X_train)
X_train = X_train.reshape(-1, 1)
x_rus, y_rus = rus.fit_resample(X_train, y_train)
x_rus = [item[0] for item in x_rus]
x_rus = np.array(x_rus).astype(np.float32)

In [7]:
# model.compile(
#     optimizer=keras.optimizers.RMSprop(),  # Optimizer
#     # Loss function to minimize
#     loss=keras.losses.SparseCategoricalCrossentropy(),
#     # List of metrics to monitor
#     metrics=[keras.metrics.SparseCategoricalAccuracy()],
# )
model = nn_model.create_baseline()
model.compile(
    optimizer=keras.optimizers.RMSprop(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

In [8]:
# X_test_n, y_test_n = ready_data(X_test, y_test)

In [9]:
# history = model.fit(x_rus, y_rus, batch_size=64, epochs=250, validation_data=(X_test_n, y_test_n))
history = model.fit(x_rus, y_rus, batch_size=64, epochs=250)

Epoch 1/250
297/297 [==============================] - 1s 2ms/step - loss: 0.6640 - sparse_categorical_accuracy: 0.6014
Epoch 2/250
297/297 [==============================] - 0s 2ms/step - loss: 0.6288 - sparse_categorical_accuracy: 0.6493
Epoch 3/250
297/297 [==============================] - 0s 2ms/step - loss: 0.6188 - sparse_categorical_accuracy: 0.6612
Epoch 4/250
297/297 [==============================] - 1s 2ms/step - loss: 0.6126 - sparse_categorical_accuracy: 0.6678
Epoch 5/250
297/297 [==============================] - 1s 2ms/step - loss: 0.6100 - sparse_categorical_accuracy: 0.6679
Epoch 6/250
297/297 [==============================] - 1s 3ms/step - loss: 0.6039 - sparse_categorical_accuracy: 0.6769
Epoch 7/250
297/297 [==============================] - 1s 3ms/step - loss: 0.6028 - sparse_categorical_accuracy: 0.6726
Epoch 8/250
297/297 [==============================] - 1s 2ms/step - loss: 0.5976 - sparse_categorical_accuracy: 0.6824
Epoch 9/250
297/297 [===================

In [12]:
# dpm.load_test()

In [14]:
# dpm.load_test()
# test_data = dpm.test_set
# X_test = task_one_data(data, labels=False)

In [13]:
# Prepare testing data.
X_test_n, y_test_n = ready_data(X_train, y_train)

predictions = model.predict(X_test_n)
predictions = [item.argmax() for item in predictions]
y_test_n = list(y_test_n)
print("Accuracy", accuracy_score(y_test_n, predictions))
print("Precision", precision_score(y_test_n, predictions, average=None))
print("Recall", recall_score(
    y_test_n, predictions, labels=[0, 1], average=None))

Accuracy 0.8518483140701117
Precision [0.99059057 0.3833612 ]
Recall [0.8443436  0.92346425]


In [321]:
# helper function to save predictions to an output file
def labels2file(p, outf_path):
	with open(outf_path,'w') as outf:
		for pi in p:
			outf.write(','.join([str(k) for k in pi])+'\n')

In [18]:
import os

In [27]:
predictions = np.array(predictions).reshape(-1,1)

In [29]:
labels2file(predictions, os.path.join('res/', 'task1.txt'))

In [ ]:
# 250 epochs
# ll = combine(data["embeddings_feature"],data["pos_embeddings"])
# ll = combine(ll,data["word_size_embeddings"])
# pos vec size = 10
# Accuracy 0.8414517669531996
# Precision [0.93634841 0.28052805]
# Recall [0.88496042 0.42713568]

In [ ]:
# 250 epochs
# ll = combine(data["embeddings_feature"],data["pos_embeddings"])
# ll = combine(ll,data["most_common_words_embeddings"])
# ll = combine(ll,data["word_size_embeddings"])
# pos vec size = 10
# Accuracy 0.8481375358166189
# Precision [0.93443526 0.28673835]
# Recall [0.89498681 0.40201005]

In [ ]:

# 500 epochs
# ll = combine(data["embeddings_feature"],data["pos_embeddings"])
# ll = combine(ll,data["most_common_tags_embeddings"])
# ll = combine(ll,data["most_common_words_embeddings"])
# ll = combine(ll,data["embeddings_feature_v2"])
# Accuracy 0.8046800382043935
# Precision [0.94813028 0.25917431]
# Recall [0.82955145 0.5678392 ]

In [ ]:
# get_text_feature : n=[3,7]
# ll = combine(data["embeddings_feature"],data["pos_embeddings"])
# ll = combine(ll,data["most_common_tags_embeddings"])
# ll = combine(ll,data["most_common_words_embeddings"])
# epochs = 250
# tag size = 50
# tags == words == 3
# Accuracy 0.8357211079274116
# Precision [0.9408755  0.28358209]
# Recall [0.87335092 0.47738693]

In [ ]:
# embeddings feature gives high recall for both
# low precision for NPCL.
# ll = combine(data["embeddings_feature"],data["pos_embeddings"])
# ll = combine(ll,data["most_common_tags_embeddings"])
# ll = combine(ll,data["most_common_words_embeddings"])
# tags == words == 3 most common.
# tagsize = 100
# Accuracy 0.720152817574021
# Precision [0.96254417 0.21502209]
# Recall [0.71873351 0.73366834]

# Task 2

dpm.load_task2()

In [88]:
dpm.load_task2()

Map of label to numerical label:
{'Unbalanced_power_relations': 0, 'Shallow_solution': 1, 'Presupposition': 2, 'Authority_voice': 3, 'Metaphors': 4, 'Compassion': 5, 'The_poorer_the_merrier': 6}


In [89]:
data = dpm.train_task2_df

In [101]:
data["keyword"].value_counts().keys()

Index(['homeless', 'in-need', 'poor-families', 'hopeless', 'refugee',
       'disabled', 'vulnerable', 'women', 'migrant', 'immigrant'],
      dtype='object')

In [105]:
data["text"][0]

'we also know that they can benefit by receiving counseling from someone who can help them understand that their feelings are normal and that their situation is not hopeless ; someone who can help them put their situation in perspective and help them communicate with others who could provide support ; someone knowledgeable about resources they can access ; someone who can help them plan for their needs and the needs of their child by developing either a parenting plan or an adoption plan .'

In [153]:
def get_text_for(data,label = 0):
    """
    Returns text that corresponds to the label as a single string.
    """
    text = []
    for i in range(len(data["text"])):
        if data["label"][i][label] == 1:
            text.append(str(label))
            z = data["text"][i]
            z = z.split(" ")
            for i in z:
                text.append(i)
                text.append(str(label))
    return " ".join(text)

In [160]:
text_cat = []

for i in range(7):
    text_cat.append(get_text_for(data,i).split(" "))

In [161]:
# text_cat = " ".join(text_cat)


In [162]:
wvec = mwv.Word2VecModelTrainer(sentences=text_cat, path="label.wordvectors")
wvec.train(size=50)
wvp = wvec.load_trained("label.wordvectors")

In [178]:
z = [wvp.distance("we","0"), wvp.distance("we","1"), wvp.distance("we","2"), wvp.distance("we","3"), wvp.distance("we","4"), wvp.distance("we","5"), wvp.distance("we","6")]

In [187]:
z = np.array(z)

In [188]:
z.mean()

0.9609289217208113

In [190]:
np.where(z<z.mean())

(array([0, 1, 3, 6]),)

In [182]:
z.index(min(z))

1

In [317]:
def predict(text, pl=84):
    """
    Returns a list of predictions for the text.
    """
    text = text.split(" ")
    pred = []
    count_dict = {0:0,1:0,2:0,3:0,4:0,5:0,6:0}
    labels = ["0","1","2","3","4","5","6"]
    for i in text:
        z = [wvp.distance(i,val) for val in labels]
        z = np.array(z)
        res = np.where(z<z.mean())[0]
        for i in res:
            count_dict[i] += 1
    # return count_dict
    
    res=  np.array(list(count_dict.values()))
    nres = np.zeros(7)
    # return res
    nres[np.where(res>np.percentile(res,pl))] = 1
    return nres

In [294]:
z = predict(data["text"][18],pl=82)

In [297]:
z

array([1., 0., 1., 0., 0., 0., 0.])

In [301]:
zx = data["label"][0] == z

In [304]:
len(np.where(zx == False)[0])

2

In [276]:
def calculate_hit(y_true, y_pred):
    hits = 0
    misses = 0
    for i in range(len(y_true)):
        res = y_true[i] == y_pred[i]
        if all(res):
            hits += 1
        else:
            misses += 1

    return hits, misses

            

In [314]:
def calculate_hit_partial(y_true, y_pred):
    hits = 0
    misses = 0
    for i in range(len(y_true)):
        res = y_true[i] == y_pred[i]
        
        misses += len(np.where(res == False)[0])

    return misses

In [315]:
losses =[]
for pl in range(75, 100):
    y_pred = [predict(data["text"][i],pl=pl) for i in range(len(data["text"]))]
    losses.append((pl,calculate_hit_partial(data["label"],y_pred)))

In [331]:
y_pred = [predict(data["text"][i],pl=84) for i in range(len(data["text"]))]

In [334]:
y_pred = [i.astype(int) for i in y_pred]

In [335]:
labels2file(y_pred, os.path.join('res/', 'task2.txt'))

In [328]:
labels2file(dpm.train_task1_df.label.apply(lambda x:[x]).tolist(), os.path.join('ref/', 'task1.txt'))

In [329]:
labels2file(dpm.train_task2_df.label.tolist(), os.path.join('ref/', 'task2.txt'))

In [336]:
! python evaluation.py . .

In [337]:
! cat scores.txt

task1_precision:0.3833612040133779
task1_recall:0.9234642497482377
task1_f1:0.5418020679468242
task2_unb:0.459119496855346
task2_sha:0.4518518518518519
task2_pre:0.6141078838174274
task2_aut:0.5051546391752577
task2_met:0.39097744360902253
task2_com:0.04583333333333334
task2_the:0.29032258064516125
task2_avg:0.3939096041839143


In [339]:
! zip submission.zip task1.txt task2.txt

	zip warning: name not matched: task2.txt
updating: task1.txt (deflated 92%)
